In [1]:
#importing libraries
import csv
import os
from googleapiclient.discovery import build
import pandas as pd
# API Key generated from the Youtube API console 
api_key = "Please enter your youtube API Key" #(https://developers.google.com/youtube/v3/getting-started)
# Establishing connection with the YouTube API key
youtube  = build('youtube','v3',developerKey=api_key)
# Setting up the directory location e.g. 
os.chdir('C:/Users/dominik/Documents/GitHub/Chess')

In [2]:
# Part 1 of the code
# This function helps us to fetch the Youtube channel playlist data
def youtube_playlist_data(id):
    token = None
    # Using the API's list function to retrive the channel data
    y_data = youtube.channels().list(id=id,part='contentDetails').execute()
    # Retrieving the "uploads" playlist Id from the channel
    youtube_playlist_id = y_data['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    video_data = []
    # The while loop which continues until the items are present in the playlist
    while 1:
        y_playlist_data = youtube.playlistItems().list(playlistId = youtube_playlist_id, part = 'snippet', maxResults = 50, pageToken = token).execute()  #Retrieving the playlist items snippet with a max result of 50 in each iteration
        video_data = video_data + y_playlist_data['items']
        # Update the token so as to get the next data
        token = y_playlist_data.get('nextPageToken')
        # If there is no token break the loop
        if token is None:
            break
    # Return the final collected data
    return video_data

In [3]:
# Part 2 of the code
# Here we pass the channel id of which the data needs to be retrieved
y_video_data = youtube_playlist_data('UCL5YbN5WLFD8dLIegT5QAbA')
# Initializing the title variable
title = []
# Initializing the description variable
description = []
# Initializing the video_id
video_id = []
# Initializing the thumbnail_default variable
thumbnail_default = []
# Initializing the thumbnail_standard variable
thumbnail_standard = []
# iterating through videos data one by one
for data in y_video_data:
    # Retrieving and appending the video title
     title.append(data['snippet']['title'])
    # Retrieving and appending the description
     description.append(data['snippet']['description'])
    # Retrieving and appending the video_id
     video_id.append("https://www.youtube.com/watch?v=" + data['snippet']['resourceId']['videoId'])   
    # Check whether the thumbnail attribute is present
     if 'thumbnails' in data['snippet'].keys():
        if 'default' in data['snippet']['thumbnails'].keys():
            # If thumbnail default present append the data
            thumbnail_default.append(data['snippet']['thumbnails']['default']['url'])
        else:
            # If thumbnail default not present append 'Null'
            thumbnail_default.append('Null')
        if 'standard' in data['snippet']['thumbnails'].keys():
            # If thumbnail standard present append the data
            thumbnail_standard.append(data['snippet']['thumbnails']['standard']['url'])
        else:
            # If thumbnail standard not present append 'Null'
            thumbnail_standard.append('Null')
     else:
         thumbnail_default.append('Null')
         thumbnail_standard.append('Null')
final_data = {'video_title': title,'video_id':video_id, 'Description':description, 'thumbnail_default': thumbnail_default, 'thumbnail_standard':thumbnail_standard} # Merge the data to form the final dataset

In [6]:
#import chess openings
openings = pd.read_csv('C:/Users/dominik/Documents/GitHub/Chess/chess_openings.csv')
openings["moves"].iloc[1]
file = pd.DataFrame(final_data)

In [7]:
#function to cluster videos of channel
def definator(df,column1,column2):
    ids = list()
    for i in range(len(openings["moves"])):
        moves = openings["moves"].iloc[i]
        #return all video urls with above opening
        df[column1].str.contains(moves)
        url_list = df.loc[df[column1].str.contains(moves)][column2].tolist()
        # append video_urls to list & define if list contains videos or not
        if len(url_list) > 0: 
            ids.append(url_list)
        else:
            ids.append(None)
        url_list = list()
        #increase i
        i=i+1
    return ids

In [8]:
#create list of videos & append list to dataframe
videos = definator(file,"Description","video_id")
openings["videos"] = videos

In [9]:
#clear all openings, which are not used
openings = openings.dropna()
#split video list to columns
openings2 = openings["videos"].apply(pd.Series)

In [10]:
#merge singular videos_urls & initial opening dataframe
openings_final = pd.concat([openings,openings2], axis=1)
#append counter of 
openings_final["count_videos"] = openings_final.count(axis=1)
#saving the file to csv format
openings_final.to_csv('opening_videos.csv')

In [11]:
file.to_csv('agadmator.csv')